# TCR/TCR antagonism model

Notebook to explore the revised AKPR model solutions for TCR/TCR antagonism. This notebook focuses on plotting the different variables at steady-state for the best parameter fits, without comparing to data for the moment. To manually vary parameters and see the effects on antagonism compared to data, use the notebook `secondary_scripts/manual_tcr_tcr_fitting_tests.ipynb`. 

## Model equations
Solving for steady-state, we find the following expressions for the numbers of bound TCRs in various proofreading stages $n$ for each type of antigen and the numbers of activated inhibitory molecules $I$. 

The numbers of receptors in each proofreading state are, for each antigen $l$ of strength $\tau_l$,

$$ C_{n,l} = \frac{R_{b,l}}{\varphi^T \tau_l + 1} \Phi_l^n \quad (0 \leq n < N^T - f^T) $$
$$ C_{n,l} = \frac{R_{b,l}}{\psi(I) \tau_l + 1} \Phi_l^{N^T - f^T} \Phi_{I,l}^{n - N^T + f^T} \quad (N^T - f^T \leq n < N^T) $$
$$ C_{N^T,l} = R_{b,l} \Phi_l^{N^T - f^T} (\Phi_{I,l})^{f^T}   $$

where we defined the regular and inhibited proofreading factors

$$ \Phi_l = \left( \frac{\varphi \tau_l}{\varphi \tau_l + 1}\right) $$ 
$$ \Phi_{I,k} = \left( \frac{\psi(I) \tau_l}{\psi(I) \tau_l + 1}\right) $$

with 

$$ \psi(I) = \varphi \frac{(I_{\mathrm{th}})^{k_I}}{(I_{\mathrm{th}})^{k_I} + I)^{k_I}} + \psi_0 \,\, .$$

The inhibitory molecule $I$ is activated out of a total pool of inhibitory molecules $I_{\mathrm{tot}}$ by the complexes $C_{m, l}$, such that

$$ I = I_{tot} \frac{\sum_l C_{m, l} / C_{m, \mathrm{th}}}{1 + \sum_l C_{m, l} / C_{m, \mathrm{th}}}  \,\,. 
$$ 

The total numbers of bound receptors are derived by expressing $R_{b, 2}$ in terms of $R_{b, 1}$, 
$$ R_{b, 2} = R_{\mathrm{tot}} - R_{b, 1} - \frac{R_{b, 1}}{\kappa \tau_1(L_1 - R_{b, 1})} $$
and solving the resulting cubic equation for $R_{b, 1}$, 
$$ p_0 ({R_{b, 1}})^3 + p_1 ({R_{b, 1}})^2 + p_2 {R_{b, 1}} + p_3 = 0 $$
where the coefficients are
$$ p_0 = \frac{\tau_1}{\tau_2} - 1 $$
$$ p_1 = -\left(\frac{\tau_1}{\tau_2} - 1 \right) \left( R_{\mathrm{tot}} + L_1 + \frac{1}{\kappa \tau_1} \right) - L_2 - \frac{\tau_1}{\tau_2} L_1 $$
$$ p_2 = \frac{\tau_1}{\tau_2} L_1^2 + \left(2\frac{\tau_1}{\tau_2} - 1 \right) R_{\mathrm{tot}} L_1 + \frac{L_1}{\kappa \tau_2} + L_1 L_2 $$
$$ p_3 = -\frac{\tau_1}{\tau_2} R_{\mathrm{tot}} {L_1}^2 $$
The physically correct solution is the only root satisfying $0 \leq R_{b, 1} < L_1$. It always exists and ensures $0 \leq R_{b, 2} < L_2$ and $R_{b,1} + R_{b,2} < R_{\mathrm{tot}}$ too. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
import json, h5py
import os

In [2]:
from models.akpr_i_model import steady_akpr_i_2ligands, steady_akpr_i_1ligand, psi_of_i
from models.kpr import steady_kpr_1ligand
from models.tcr_car_akpr_model import steady_akpr_i_receptor_types
from mcmc.costs_tcr_car_antagonism import repackage_tcr_car_params
from utils.preprocess import michaelis_menten
from mcmc.costs_tcr_car_antagonism import repackage_tcr_car_params
from mcmc.utilities_tcr_car_antagonism import load_tcr_tcr_akpr_fits

In [3]:
# Aesthetic parameters. Small scale figure.
scaleup = 1.0
do_save = False

# rcParams
plt.rcParams["font.size"] =  8. * scaleup  # Default, smallish
plt.rcParams["figure.dpi"] = 150.0
plt.rcParams["figure.figsize"] = (3, 3)
plt.rcParams["axes.labelpad"] = 0.5 * scaleup
plt.rcParams["axes.linewidth"] = 0.75 * scaleup     # edge line width
plt.rcParams["lines.linewidth"] = 2.0 * scaleup               # line width in points
plt.rcParams["lines.markersize"] = 2.5 * scaleup   # marker size, in points
for x in ["xtick.", "ytick."]:
    plt.rcParams[x + "major.size"] = 2.25 * scaleup
    plt.rcParams[x + "minor.size"] = 1.8 * scaleup
    plt.rcParams[x + "major.pad"] = 2.0 * scaleup
    plt.rcParams[x + "minor.pad"] = 1.9 * scaleup
    plt.rcParams[x + "minor.width"] = 0.5 * scaleup
    plt.rcParams[x + "major.width"] = 0.75 * scaleup
plt.rcParams["axes.spines.top"] = False
plt.rcParams["axes.spines.right"] = False

# Display figures larger
plt.rcParams['figure.dpi'] = 150 # default for me was 75

#plt.rcParams['font.family'] = 'Helvetica'
# TODO: use font manager API: https://matplotlib.org/stable/api/font_manager_api.html
# to import Helvetica.ttc from the data/ folder? So it's robust on Google Drive too. 
# Not easy to get Helvetica.ttf which matplotlib would support (doesn't seem very legit). 
plt.rcParams["font.family"] = "Arial"
plt.rcParams["pdf.fonttype"] = 42  # truetype, prevents corruption of editable text in figures. 

## Model parameters
Fitted values on TCR-TCR antagonism. We use $(k, m, f)= (1, 4, 1)$ since it provides the best fit. 

In [4]:
with open(os.path.join("data", "pep_tau_map_ot1.json"), "r") as handle:
    pep_tau_map = json.load(handle)

In [5]:
# Load best parameter fit
results_fname = "results/mcmc/mcmc_results_akpr_i.h5"
analysis_res_fname = "results/mcmc/mcmc_analysis_akpr_i.json"
with open(analysis_res_fname, "r") as jfile:
    all_results_dicts = json.load(jfile)
    del jfile

# Import parameter fits in linear scale (load function undoes log(parameters))
tcr_loads = load_tcr_tcr_akpr_fits(results_fname, analysis_res_fname)
# params: phi, kappa, cmthresh, I0p, kp, psi0, gamma_tt
# Then, [N, m, f] and I_tot
tcr_params, tcr_nmf, tcr_itot = tcr_loads
tcr_params = tcr_params[:-1]  # drop gamma_tt = 1, useless for TCR/TCR

# Compare to psi = 0 later on
tcr_params0 = np.asarray(tcr_params).copy()
tcr_params0[-1] = 0.0

# Load constant parameter values
samples_fname = results_fname
with h5py.File(samples_fname, "r") as rfile:
    data_group = rfile.get("data")
    fit_param_names = list(rfile.get("samples").attrs.get("param_names"))
    l_conc_mm_params = data_group.get("l_conc_mm_params")[()]
    # For TCR-TCR: ["rates_others", "total_RI", "N", "tau_agonist"]
    cost_args_loaded = [data_group.get(a)[()]
                        for a in data_group.attrs.get("cost_args_names")]
    del data_group, rfile
tcr_ritot = cost_args_loaded[1]  # receptor number and I_tot = 1
tau_agonist = cost_args_loaded[3]  # N4 peptide

# TCR/TCR model analysis

## Balance between antagonism and activation
Why are strong antigens not antagonizing the most despite producing more $C_m$? Why are intermediate antigens the best antagonists?

In [6]:
# Define the range of L_i, tau_i to test
tau_range = np.geomspace(0.1, 20.0, 200)
l_range = [michaelis_menten(1e-3, *l_conc_mm_params)]  # 1 nM dose
# Index each output array [l2, tau2]
df_output_akpr_1 = pd.DataFrame(0.0, 
    index=pd.MultiIndex.from_product([l_range, tau_range], names=["L", "tau"]), 
    columns=pd.Index(["C_" + str(n) for n in range(tcr_nmf[0]+1)] + ["I"], name="Variable")
)
df_output_kpr_1 = pd.DataFrame(0.0, 
    index=pd.MultiIndex.from_product([l_range, tau_range], names=["L", "tau"]), 
    columns=pd.Index(["C_" + str(n) for n in range(tcr_nmf[0]+1)], name="Variable")
)

for i in range(tau_range.shape[0]):  # over tau
    taup = tau_range[i]
    for j in range(len(l_range)):  # over L
        lp = l_range[j]
        # ratesp, tau1p, L1p, ri_tots, nmf, large_l=False
        # phip, kappap, Csp, I0p, kp, psi0 = ratesp
        # Rp, ITp = ri_tots
        # Np, mp, fp = nmf
        df_output_akpr_1.loc[(lp, taup), :] = steady_akpr_i_1ligand(
            tcr_params, taup, lp, tcr_ritot, tcr_nmf)
        df_output_kpr_1.loc[(lp, taup), :] = steady_kpr_1ligand(
            tcr_params[:2], taup, lp, tcr_ritot[0], tcr_nmf[0])

In [7]:
def loglog_scaling_law(xrange, x0, y0, n):
    return y0 * (xrange/x0)**n  # in linear scale

def get_angle(xvals, yvals, xextent, yextent):
    slope = (yvals[-1]-yvals[0]) / (xvals[-1] - xvals[0])
    extent_slope = (yextent[-1]-yextent[0])/(xextent[-1]-xextent[0])
    return np.arctan(slope / extent_slope) * 360 / (2.0*np.pi)

In [ ]:
fig, ax = plt.subplots()

# C_m, C_N from AKPR line
cn_lbl = "C_" + str(tcr_nmf[0])
cm_lbl = "C_" + str(tcr_nmf[1])

# Colors from model diagrams. CAR color: (41.0, 102.0, 142.0)
c_palette = {"C_m": "xkcd:sky blue", "C_N": np.asarray((69.0, 153.0, 84.0))/255.0, 
             "I": np.asarray((117.0, 26.0, 124.0))/255.0}

# KPR and AKPR C_m
#ax.plot(tau_range, df_output_kpr_1.loc[(l_range[0], tau_range), cm_lbl].values, 
#        label=r"Pure KPR $C_m$", ls="--", zorder=1)
ax.plot(tau_range, df_output_akpr_1.loc[(l_range[0], tau_range), cm_lbl].values, 
        label=r"Intermediate complex $C_m$", zorder=0, color=c_palette["C_m"])

# KPR and AKPR C_N
#ax.plot(tau_range, df_output_kpr_1.loc[(l_range[0], tau_range), cn_lbl].values, 
#        label=r"Pure KPR $C_N$", ls="--", zorder=3)
ax.plot(tau_range, df_output_akpr_1.loc[(l_range[0], tau_range), cn_lbl].values, 
        label=r"Output $C_N$", zorder=2, color=c_palette["C_N"])

# And finally, I in revised AKPR
ax.plot(tau_range, df_output_akpr_1.loc[(l_range[0], tau_range), "I"].values, 
        label="Inhibitory species $I$", zorder=4, color=c_palette["I"])

# Annotate the scalings: tau^m for C_m and I, tau^N at first, then tau^N-m for C_N
# Use log-scale before plotting scaling laws
ax.set(xlabel=r"TCR antigen strength, $\tau^T$ (s)", ylabel="Model variable", 
       xscale="log", yscale="log")
xlims_arr = np.asarray(ax.get_xlim())
ylims_arr = np.asarray(ax.get_ylim())

# C_N early scaling
sl = slice(10, 70, 1)
tau_slope = tau_range[sl]
scale_slope = loglog_scaling_law(tau_slope, tau_slope[0], 3e-8, tcr_nmf[0])
orient = get_angle(np.log(tau_slope), np.log(scale_slope), 
                   np.log(xlims_arr), np.log(ylims_arr))
ax.plot(tau_slope, scale_slope, color=c_palette["C_N"], ls="--", lw=1.0)
#ax.plot(tau_range[sl], df_output_kpr_1.loc[(l_range[0], tau_range[sl]), cn_lbl]*10.0, 
#        color="k", ls="--", lw=1.0)
ax.annotate(r"$C_N \sim L \tau^N$", xy=(tau_slope[0], scale_slope[0]*10.0), 
            rotation=orient, color=c_palette["C_N"])

# C_m scaling
scale_slope = loglog_scaling_law(tau_slope, tau_slope[0], 1e-4, tcr_nmf[1])
orient = get_angle(np.log(tau_slope), np.log(scale_slope), 
                   np.log(xlims_arr), np.log(ylims_arr))
ax.plot(tau_slope, scale_slope, color=c_palette["C_m"], ls="--", lw=1.0)
ax.annotate(r"$C_m \sim L \tau^m$", xy=(tau_slope[0], scale_slope[0]*5.0), 
            rotation=orient, color=c_palette["C_m"])

# I scaling
sl = slice(40, 100, 1)
tau_slope = tau_range[sl]
scale_slope = loglog_scaling_law(tau_slope, tau_slope[0], 5e-8, tcr_nmf[1])
orient = get_angle(np.log(tau_slope), np.log(scale_slope), 
                   np.log(xlims_arr), np.log(ylims_arr))
ax.plot(tau_slope, scale_slope, color=c_palette["I"], ls="--", lw=1.0)
ax.annotate(r"$I \sim L \tau^m$", xy=(tau_slope[-1], scale_slope[-1]*0.2), ha="right",
            rotation=orient, color=c_palette["I"], va="top")

# C_N late scaling
sl = slice(120, 180, 1)
tau_slope = tau_range[sl]
scale_slope = loglog_scaling_law(tau_slope, tau_slope[0], 5e-2, tcr_nmf[0]-tcr_nmf[1])
orient = get_angle(np.log(tau_slope), np.log(scale_slope), 
                   np.log(xlims_arr), np.log(ylims_arr))
ax.plot(tau_slope, scale_slope, color=c_palette["C_N"], ls="--", lw=1.0)
ax.annotate(r"$C_N \sim \tau^{N-m}$", xy=(tau_slope[0], scale_slope[0]*3.0), 
            rotation=orient, color=c_palette["C_N"])

handles, labels = ax.get_legend_handles_labels()
leg_kwargs = dict(frameon=False, fontsize=7)

# Create a legend for the first line.
first_legend = ax.legend(handles[:1], labels[:1], loc="upper left", **leg_kwargs)

# Add the legend manually to the Axes.
ax.add_artist(first_legend)

# Create another legend for the second line.
ax.legend(handles[1:], labels[1:], loc="lower right", **leg_kwargs)
fig.tight_layout()
if do_save:
    fig.savefig("figures/model_details/revised_akpr_single-antigen_scalings.pdf", 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

## Single antigen response curves
There are three regimes in the response curves of the model. 

- At small $L$, there is a pure kinetic proofreading regime while $I \approx 0$ and receptors are unsaturated: $C_N \approx R_b \Phi^N \sim L \tau^N$. This is the region where curves are increasing linearly with $L$ in the log-log plot. 
	
- Then, since the best parameter fit has a small threshold $I_{\mathrm{th}}$ for saturation of the regulated rate $\psi(I)$, but a large threshold $C_{m, \mathrm{th}}$ for the activation of $I$ by $C_m$, the first thing occurring as $L$ increases is that $I$ reaches $I_{\mathrm{th}}$ while $C_m \ll C_{m, \mathrm{th}}$ still. Then, $I \approx I_{\mathrm{tot}} C_m / C_{m, \mathrm{th}} \sim L \tau^m$, while $\psi(I) \approx \varphi I_{\mathrm{th}} / I$, such that $C_N \approx R_b \Phi^{N-1} \psi(I) \tau \sim L \tau^N / L \tau^m \sim \tau^{N-m}$. This is the region where $C_N$ as a function of $L$ is flat. Note that this regime starts at a $\tau$-dependent $L^*$, when $I_{\mathrm{th}} = I$, implying that $L^* \sim \tau^{-m}$ -- hence, this happens earlier for stronger antigens. 
	
- Eventually, $C_m$ reaches $C_{m, \mathrm{th}}$, so the inhibitory molecule $I$ saturates near $I_{tot}$, $\psi(I) \approx \varphi I_{\mathrm{th}} / I_{\mathrm{tot}}$, and thus $C_N \sim L \tau^N \times I_{\mathrm{th}} / I_{\mathrm{tot}}$: another KPR scaling regime but at a lower amplitude, reduced by a factor $I_{\mathrm{th}} / I_{\mathrm{tot}} < 1$. This is the rightmost region where response curves start increasing again (before receptors saturate). Note that this occurs at another ligand quantity $L^{**} > L^*$, but also scaling as $L^{**} \sim \tau^{-m}$. Weaker ligands thus never reach this second threshold and do not benefit from much extra activation at high doses. 


We draw regime separation thresholds on the plots. We compute $C_N$ at $L^*$ and $L^{**}$ for every $\tau$. 

In [9]:
def compute_transition_ls(ratesp, taup, ri_tots, nmf):
    phip, kappap, cmth, I0p, kp, psi0 = ratesp
    Rp, ITp = ri_tots
    Np, mp, fp = nmf
    if Np - fp <= mp: 
        raise ValueError("I have not computed the implicit case solution")
    if I0p / ITp > 0.1:
        raise ValueError("I have not computed cases where I_{th} is large")

    # KPR to absolute transition: when I = I_th
    cmstar = cmth * I0p/ITp / (1.0 - I0p/ITp)
    kappa_fact = kappap * Rp * taup / (kappap * Rp * taup + 1.0)
    phifact = phip * taup / (phip*taup + 1.0)
    lstar = cmstar / kappa_fact * (phip * taup + 1.0) / phifact**mp
    # Let's take C_m = C_{m, th} / 2 instead
    lstar2 = cmth / 2.0 / kappa_fact * (phip * taup + 1.0) / phifact**mp
    return lstar, lstar2

In [10]:
# Define the range of L_i, tau_i to test
tau_range = np.asarray([10.0, 7.0, 5.0, 3.0, 2.0])
l_range = np.logspace(0, 5, 201)
# Index each output array [l2, tau2]
output = np.zeros([tau_range.size, l_range.size])
for i in range(output.shape[0]):  # over tau
    taup = tau_range[i]
    for j in range(output.shape[1]):  # over L
        lp = l_range[j]
        # ratesp, tau1p, L1p, ri_tots, nmf, large_l=False
        # phip, kappap, Csp, I0p, kp, psi0 = ratesp
        # Rp, ITp = ri_tots
        # Np, mp, fp = nmf
        output[i, j] = steady_akpr_i_1ligand(tcr_params, taup, lp, tcr_ritot, tcr_nmf)[tcr_nmf[0]]

# Compute transition line and C_N at every point of that line. 
tau_range2 = np.linspace(tau_range.min()*0.8, tau_range.max()*2, 101)
lstar_range, lstar2_range = compute_transition_ls(tcr_params, tau_range2, tcr_ritot, tcr_nmf)
cn_at_lstar = np.zeros(lstar_range.shape)
cn_at_lstar2 = np.zeros(lstar_range.shape)
for k in range(tau_range2.shape[0]):
    results_akpr = steady_akpr_i_1ligand(tcr_params, tau_range2[k], 
                                lstar_range[k], tcr_ritot, tcr_nmf)
    results_akpr2 = steady_akpr_i_1ligand(tcr_params, tau_range2[k], 
                                lstar2_range[k], tcr_ritot, tcr_nmf)
    cn_at_lstar[k] = results_akpr[tcr_nmf[0]]
    cn_at_lstar2[k] = results_akpr2[tcr_nmf[0]]

In [ ]:
fig, ax = plt.subplots()
for i in range(output.shape[0]):  # over tau
    ax.plot(l_range, output[i], label=r"$\tau = {:.0f}$ s".format(tau_range[i]))
ax.set(xscale="log", yscale="log", xlabel=r"Antigen quantity $L$", ylabel=r"Steady-state $C_N$")
ylims = ax.get_ylim()
xlims = ax.get_xlim()
ax.plot(lstar_range, cn_at_lstar, label=r"$I = I_{\mathrm{th}}$", color="k", ls="--", lw=1.5)
ax.plot(lstar2_range, cn_at_lstar2, label=r"$C_m = C_{m, \mathrm{th}}$", color="grey", ls=":", lw=1.5)
ax.set_ylim(*ylims)
ax.set_xlim(*xlims)
ax.legend(ncol=2, fontsize=9)
fig.set_size_inches(4.5, 3.1)
fig.tight_layout()
if do_save:
    fig.savefig("figures/model_details/revised_akpr_output_vs_l_various_taus.pdf", 
        transparent=True, bbox_inches="tight")
plt.show()
plt.close()

## Model variables in response to mixtures of antigens and antagonism
Plot the model solution beyond just the antagonism ratio: plot also the level of inhibitory species $I$, the final step(s) proofreading rate $\psi(I)$, etc. 
We explicitly compute the model solutions within the notebook rather than using model panel functions. 

In [ ]:
# Compute all complexes and the inhibitory species as a function of tau_tcr and L_TCR 
# Also plot the psi functions. See how all that stuff varies and saturates. 
tau_tcr_range = np.linspace(0.001, 10.0, 101)
l2_conc_range = np.asarray([1e-5, 1e-4, 1e-3, 1e-2, 1e0])
conc_agonist =  1e-5  # 10 pM = 1e-5 in uM
l_tcr_range = michaelis_menten(l2_conc_range, *l_conc_mm_params)
l_agonist = michaelis_menten(conc_agonist, *l_conc_mm_params)
# ithresh, ki, phip, psi0
psi_fct_params = tcr_params[3:5] + tcr_params[0:1] + tcr_params[5:6]
print(l_agonist)

agonist_alone = steady_akpr_i_1ligand(tcr_params, tau_agonist, l_agonist, tcr_ritot, tcr_nmf)

model_columns = pd.Index([r"$C_{" + str(n) + r",1}$" for n in range(tcr_nmf[0]+1)] 
                   + [r"$C_{" + str(n) + r",2}$" for n in range(tcr_nmf[0]+1)]
                   + [r"$I$", r"$\psi(I)$", "Ratio"], name="Variable")
model_index = pd.MultiIndex.from_product([l_tcr_range, tau_tcr_range], names=[r"$L_2$", r"$\tau_2$"])
df_mix = pd.DataFrame(np.zeros([len(model_index), len(model_columns)]), 
                       columns=model_columns, index=model_index)

antag_alone_columns = pd.Index([r"$C_{" + str(n) + r",2}$" for n in range(tcr_nmf[0]+1)] 
                               + [r"$I$", r"$\psi(I)$"], name="Variable")
df_antag_alone = pd.DataFrame(np.zeros([len(model_index), len(antag_alone_columns)]), 
                       columns=antag_alone_columns, index=model_index)

for l_2, tau_2 in model_index:
    taus = np.asarray([tau_agonist, tau_2])
    lvec = np.asarray([l_agonist, l_2])
    complexes_mix = steady_akpr_i_2ligands(tcr_params, taus, lvec, tcr_ritot, tcr_nmf)
    complexes_agonist = complexes_mix[:tcr_nmf[0]+1]
    complexes_antagonist = complexes_mix[tcr_nmf[0]+1:-1]
    df_mix.loc[(l_2, tau_2), r"$C_{0,1}$":r"$C_{" + str(tcr_nmf[0]) + ",1}$"] = complexes_agonist
    df_mix.loc[(l_2, tau_2), r"$C_{0,2}$":r"$C_{" + str(tcr_nmf[0]) + ",2}$"] = complexes_antagonist
    df_mix.loc[(l_2, tau_2), r"$I$"] = complexes_mix[-1]
    df_mix.loc[(l_2, tau_2), r"$\psi(I)$"] = psi_of_i(complexes_mix[-1], *psi_fct_params)
    output = complexes_agonist[tcr_nmf[0]] + complexes_antagonist[tcr_nmf[0]]
    df_mix.loc[(l_2, tau_2), "Ratio"] = output / agonist_alone[tcr_nmf[0]]
    
    # Also, antagonist alone at that l2, tau2
    antag_alone = steady_akpr_i_1ligand(tcr_params, tau_2, l_2, tcr_ritot, tcr_nmf)
    df_antag_alone.loc[(l_2, tau_2), r"$C_{0,2}$":r"$C_{" + str(tcr_nmf[0]) + ",2}$"] = antag_alone[:-1]
    df_antag_alone.loc[(l_2, tau_2), "$I$"] = antag_alone[-1]
    df_antag_alone.loc[(l_2, tau_2), r"$\psi(I)$"] = psi_of_i(antag_alone[-1], *psi_fct_params)

In [ ]:
# Plot with several variables compared
df_plot = df_mix + df_mix.max(axis=0)*1e-4
df_plot = df_plot.loc[:, r"$I$":]
df_plot = df_plot.stack().reset_index()
df_plot[r"$L_2$"] = np.round(df_plot[r"$L_2$"], 0).astype(int).astype(str)
hue_order = np.round(l_tcr_range[::-1], 0).astype(int).astype(str)

g = sns.relplot(data=df_plot, x=r"$\tau_2$", y=0, col="Variable", hue=r"$L_2$", 
               height=2.0, col_wrap=4, kind="line", facet_kws=dict(sharey=False), 
               hue_order=hue_order, palette="magma")
for ax in g.axes.flat:
    ax.set_yscale("log")
    
# Now, antagonist alone below, for comparison
df_plot = df_antag_alone + df_antag_alone.max(axis=0)*1e-4
df_plot = df_plot.loc[:, r"$I$":]
df_plot = df_plot.stack().reset_index()
df_plot[r"$L_2$"] = np.round(df_plot[r"$L_2$"], 0).astype(int).astype(str)

g = sns.relplot(data=df_plot, x=r"$\tau_2$", y=0, col="Variable", hue=r"$L_2$", 
               height=2.0, col_wrap=4, kind="line", facet_kws=dict(sharey=False), 
               hue_order=hue_order, palette="magma")
for ax in g.axes.flat:
    ax.set_yscale("log")

plt.show()
plt.close()

## Antagonist output alone vs in presence of the agonist
We have absolute discrimination for single antigens. Do we still have it in presence of the agonist? In other words, is the activation of the inhibitory molecule $I$ by the agonist enough to push the response to the final increasing part of the curve (3rd regime)? Or is the response curve still flat?

In [ ]:
# Plot all complexes and all SHP-1s as a function of tau_tcr and L_TCR in presence of CD19. 
# Also plot the psi functions. See how all that stuff varies and saturates. 
tau_tcr_range2 = np.asarray([2.5, 4.0, 5.0, 7.0])
l2_conc_range2 = np.logspace(-4, 0, 101)  # in uM
conc_agonist2 =  np.asarray([0.0, 1e-5, 1e-4])  # 10 pM = 1e-5 in uM
l_tcr_range2 = michaelis_menten(l2_conc_range2, *l_conc_mm_params)
l_agonist2 = michaelis_menten(conc_agonist2, *l_conc_mm_params)
# ithresh, ki, phip, psi0
psi_fct_params = tcr_params[3:5] + tcr_params[0:1] + tcr_params[5:6]
print(l_agonist)

model_columns2 = pd.Index([r"$C_{" + str(n) + r",1}$" for n in range(tcr_nmf[0]+1)] 
                   + [r"$C_{" + str(n) + r",2}$" for n in range(tcr_nmf[0]+1)]
                   + [r"$I$", r"$\psi(I)$"], name="Variable")
model_index2 = pd.MultiIndex.from_product([l_agonist2, l_tcr_range2, tau_tcr_range2], 
                                          names=[r"$L_1$", r"$L_2$", r"$\tau_2$"])
df_mix2 = pd.DataFrame(np.zeros([len(model_index2), len(model_columns2)]), 
                       columns=model_columns2, index=model_index2)

# Agonist alone (C_N output) at each agonist concentration
df_agonist_alone = pd.Series(np.zeros(len(conc_agonist2)), 
                    index=pd.Index(l_agonist2, name="$L_1$"))

for k in model_index2:
    l_1, l_2, tau_2 = k
    if l_1 == 0.0:
        # Also, antagonist alone at that l2, tau2
        antag_alone = steady_akpr_i_1ligand(tcr_params, tau_2, l_2, tcr_ritot, tcr_nmf)
        df_mix2.loc[k, r"$C_{0,2}$":r"$C_{" + str(tcr_nmf[0]) + ",2}$"] = antag_alone[:-1]
        df_mix2.loc[k, "$I$"] = antag_alone[-1]
        df_mix2.loc[k, r"$\psi(I)$"] = psi_of_i(antag_alone[-1], *psi_fct_params)
    else:
        taus = np.asarray([tau_agonist, tau_2])
        lvec = np.asarray([l_1, l_2])
        complexes_mix = steady_akpr_i_2ligands(tcr_params, taus, lvec, tcr_ritot, tcr_nmf)
        complexes_agonist = complexes_mix[:tcr_nmf[0]+1]
        complexes_antagonist = complexes_mix[tcr_nmf[0]+1:-1]
        df_mix2.loc[k, r"$C_{0,1}$":r"$C_{" + str(tcr_nmf[0]) + ",1}$"] = complexes_agonist
        df_mix2.loc[k, r"$C_{0,2}$":r"$C_{" + str(tcr_nmf[0]) + ",2}$"] = complexes_antagonist
        df_mix2.loc[k, r"$I$"] = complexes_mix[-1]
        df_mix2.loc[k, r"$\psi(I)$"] = psi_of_i(complexes_mix[-1], *psi_fct_params)
    agonist_alone = steady_akpr_i_1ligand(tcr_params, tau_agonist, l_1, tcr_ritot, tcr_nmf)
    df_agonist_alone.loc[l_1] = agonist_alone[tcr_nmf[0]]

In [ ]:
cn_lbl = r"$C_{" + str(tcr_nmf[0]) + ",2}$"
max_y =  df_mix2.loc[:, cn_lbl].max()*1e-12
df_plot = df_mix2.loc[:, cn_lbl] + max_y*1e-4
df_plot = df_plot.to_frame().reset_index()
df_plot["$L_1$"] = df_plot["$L_1$"].round(1)

g = sns.relplot(data=df_plot, x=r"$L_2$", y=cn_lbl, hue=r"$\tau_2$", 
               height=2.5, kind="line", style=r"$L_1$", facet_kws=dict(), 
               hue_order=tau_tcr_range[::-1].astype(str), palette="flare")
nm_um_highlights = michaelis_menten(np.asarray([1e-3, 1e0]), *l_conc_mm_params)
for ax in g.axes.flat:
    ax.set_yscale("log")
    ax.set_xscale("log")
    for i in range(len(nm_um_highlights)):
        ax.axvline(nm_um_highlights[i], ls=":", color="k")
    for i, k in enumerate(df_agonist_alone.index):
        if k == 0.0: 
            continue
        else:
            print(k, df_agonist_alone.loc[k])
            ax.axhline(df_agonist_alone.loc[k], lw=1.0+0.5*i, 
                       label="Agonist alone, $L_1 = {}$".format(k), color="k")
g.tight_layout()
plt.show()
plt.close()

What do we see? We see that because of the third regime at large $L$ where curves go up again (because $I$ is saturated), the antagonist output increases with $\tau$ faster at large $L$ than at intermediate $L$. 
There is not a significant difference in the antagonist output in the presence or absence of a few agonists. 

## Compare to $\psi_0 = 0$
To figure out if this parameter is important or not. It looks like it is not very important, but can still make a small difference in capturing enhancement properly. 

In [ ]:
# Compute all complexes and all SHP-1s as a function of tau_tcr and L_TCR in presence of CD19. 
# Also plot the psi functions. See how all that stuff varies and saturates. 
tau_tcr_range = np.linspace(0.001, 10.0, 101)
l2_conc_range = np.asarray([1e-5, 1e-4, 1e-3, 1e-2, 1e0])
conc_agonist =  1e-5  # 10 pM = 1e-5 in uM
l_tcr_range = michaelis_menten(l2_conc_range, *l_conc_mm_params)
l_agonist = michaelis_menten(conc_agonist, *l_conc_mm_params)
print(l_agonist)

agonist_alone = steady_akpr_i_1ligand(tcr_params, tau_agonist, l_agonist, tcr_ritot, tcr_nmf)
agonist_alone0 = steady_akpr_i_1ligand(tcr_params0, tau_agonist, l_agonist, tcr_ritot, tcr_nmf)

model_columns = pd.Index(["Best", "psi_0=0"])
model_index = pd.MultiIndex.from_product([l_tcr_range, tau_tcr_range], names=[r"$L_2$", r"$\tau_2$"])
df_mix_psi = pd.DataFrame(np.zeros([len(model_index), len(model_columns)]), 
                       columns=model_columns, index=model_index)


for l_2, tau_2 in model_index:
    taus = np.asarray([tau_agonist, tau_2])
    lvec = np.asarray([l_agonist, l_2])
    complexes_mix = steady_akpr_i_2ligands(tcr_params, taus, lvec, tcr_ritot, tcr_nmf)
    complexes_agonist = complexes_mix[:tcr_nmf[0]+1]
    complexes_antagonist = complexes_mix[tcr_nmf[0]+1:-1]
    output = complexes_agonist[tcr_nmf[0]] + complexes_antagonist[tcr_nmf[0]]
    df_mix_psi.loc[(l_2, tau_2), "Best"] = output / agonist_alone[tcr_nmf[0]]
    
    complexes_mix0 = steady_akpr_i_2ligands(tcr_params0, taus, lvec, tcr_ritot, tcr_nmf)
    complexes_agonist0 = complexes_mix0[:tcr_nmf[0]+1]
    complexes_antagonist0 = complexes_mix0[tcr_nmf[0]+1:-1]
    output0 = complexes_agonist0[tcr_nmf[0]] + complexes_antagonist0[tcr_nmf[0]]
    df_mix_psi.loc[(l_2, tau_2), "psi_0=0"] = output0 / agonist_alone0[tcr_nmf[0]]
df_mix_psi.columns.name = "Psi version"

In [ ]:
df_plot = df_mix_psi.stack("Psi version")
df_plot.name = "Ratio"
df_plot = df_plot.to_frame().reset_index()
df_plot[r"$L_2$"] = df_plot[r"$L_2$"].round(0).astype("int").astype("str")
print(df_plot)

g = sns.relplot(data=df_plot, x=r"$\tau_2$", hue=r"$L_2$", y="Ratio", style="Psi version", 
               height=2.5, kind="line", palette="flare")
g.tight_layout()
plt.show()
plt.close()

## Inhibitory species $I$ as a function of $\tau^T_l$

In [18]:
# Plot I as a function of tau_tcr and tau_car
# Also plot the psi functions. See how all that stuff varies and saturates. 
tau1_range = np.linspace(0.001, 10.0, 100)
tau2_range = np.linspace(0.001, 10.0, 100)
tau1_grid, tau2_grid = np.meshgrid(tau1_range, tau2_range, indexing="ij")  # tau2 on the x axis
l1_conc = 1e-4   # 100 pM agonist = 10^-4 uM. 
l2_conc = 1e0  # 1 uM antagonist
l1_tcr = michaelis_menten(l1_conc, *l_conc_mm_params)
l2_tcr = michaelis_menten(l2_conc, *l_conc_mm_params)

mat_i12 = pd.DataFrame(np.zeros(tau1_grid.shape), index=pd.Index(tau1_range, name="tau_1"), 
                       columns=pd.Index(tau2_range, name="tau_2"))
lvec = np.asarray([l1_tcr, l2_tcr])
for i in range(tau1_range.size):
    for j in range(tau2_range.size):
        taus = np.asarray([tau1_range[i], tau2_range[j]])
        complexes_mix = steady_akpr_i_2ligands(tcr_params, taus, lvec, tcr_ritot, tcr_nmf)
        mat_i12.loc[tau1_range[i], tau2_range[j]] = complexes_mix[-1]
#display(mat_i12)

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(mat_i12.values[::-1, :], cmap=plt.cm.Purples, 
    extent=(tau2_range[0], tau2_range[-1], tau1_range[0], tau1_range[-1]))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="8%", pad=0.1)
cbar = fig.colorbar(im, cax=cax)
cbar.set_label(r"$I$", labelpad=3)
ax.set(xlabel=r"$\tau^T_1$", ylabel=r"$\tau^T_2$")
fig.tight_layout()
if do_save:
    fig.savefig("figures/model_details/inhibitory_species_vs_taus_tcr-tcr.pdf", 
            transparent=True, bbox_inches="tight")
plt.show()
plt.close()